# Logistic Regression Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
import findspark

In [2]:
findspark.init('/home/vlad1974/spark-2.1.0-bin-hadoop2.7')

In [3]:
import pyspark

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName('logregconsult').getOrCreate()

In [6]:
data = spark.read.csv('customer_churn.csv',inferSchema=True,
                     header=True)

In [7]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



### Check out the data

In [8]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                null|                null|0.16666666666666666|
| stddev|         null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [9]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

### Format for MLlib

In [10]:
from pyspark.ml.feature import VectorAssembler

In [11]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Years',
 'Num_Sites'],outputCol='features')

In [12]:
output = assembler.transform(data)

In [13]:
final_data = output.select('features','churn')

### Test Train Split

In [14]:
train_churn,test_churn = final_data.randomSplit([0.7,0.3])

### Fit the model

In [15]:
from pyspark.ml.classification import LogisticRegression

In [16]:
lr_churn = LogisticRegression(labelCol='churn')

In [17]:
fitted_churn_model = lr_churn.fit(train_churn)

In [18]:
training_sum = fitted_churn_model.summary

In [19]:
training_sum.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              churn|         prediction|
+-------+-------------------+-------------------+
|  count|                616|                616|
|   mean| 0.1737012987012987| 0.1396103896103896|
| stddev|0.37916030682909496|0.34686401471598155|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



### Evaluate results

Let's evaluate the results on the data set we were given (using the test data)

In [20]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [21]:
pred_and_labels = fitted_churn_model.evaluate(test_churn)

In [22]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[28.0,11128.95,5....|    0|[4.73029705469660...|[0.99125332981674...|       0.0|
|[28.0,11204.23,3....|    0|[1.71417674684059...|[0.84737724303453...|       0.0|
|[29.0,9617.59,5.4...|    0|[4.42003915276318...|[0.98810932838132...|       0.0|
|[29.0,12711.15,5....|    0|[5.61976621708188...|[0.99638760809841...|       0.0|
|[30.0,6744.87,5.1...|    0|[3.22302546323107...|[0.96169163092315...|       0.0|
|[30.0,8874.83,5.5...|    0|[3.00681742669994...|[0.95288116686094...|       0.0|
|[30.0,10183.98,5....|    0|[3.28237342672209...|[0.96381914047232...|       0.0|
|[30.0,11575.37,5....|    1|[4.55511682502512...|[0.98959610700921...|       0.0|
|[31.0,5304.6,5.29...|    0|[3.04649944844241...|[0.95463115689257...|       0.0|
|[31.0,9574.89,7

### Using AUC

In [29]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction',
                                           labelCol='churn')

In [30]:
auc = churn_eval.evaluate(pred_and_labels.predictions)

In [31]:
auc

0.8711763002991423

### Predict on brand new unlabeled data

In [32]:
final_lr_model = lr_churn.fit(final_data)

In [33]:
new_customers = spark.read.csv('new_customers.csv',inferSchema=True,
                              header=True)

In [34]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [35]:
test_new_customers = assembler.transform(new_customers)

In [36]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [37]:
final_results = final_lr_model.transform(test_new_customers)

In [38]:
final_results.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

